In [3]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender,bm25_weight)
from sklearn.preprocessing import normalize
import numpy as np
import pickle
import scipy.sparse as sp

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [100]:
#artist
icm = dr.getICM_csr(albums=False,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_ar = urm*s.T
r_ar = normalize(r_ar, axis=0)
if evaluation: print dr.evaluateMAP(r_ar,verbose=False)
r_ar = dr.reduceRM(r_ar)

0.0702706944444


In [22]:
#album
icm = dr.getICM_csr(albums=True,artists=False,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_al = urm*s.T
r_al = normalize(r_al, axis=0)
#r_al = bm25_weight(r_al)
if evaluation: print dr.evaluateMAP(r_al,verbose=False)
r_al = dr.reduceRM(r_al)

0.0711872222222


In [15]:
#album and artist
icm = dr.getICM_csr(albums=True,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender(K=55)
model.fit(icm)
s = model.similarity
s.setdiag(0)
r_al_ar = urm*s.T #or s
#r_al_ar = normalize(r_al_ar, axis=0)
r_al_ar = bm25_weight(r_al_ar)
if evaluation: print dr.evaluateMAP(r_al_ar,verbose=False)
r_al_ar = dr.reduceRM(r_al_ar)

0.0832627777778


In [ ]:
0.0807190277778

In [13]:
#tags
icm = dr.getICM_csr(albums=False,artists=False,tags=True)
#icm = normalize(icm,axis=1)   # little improvment
model = BM25Recommender(K = 15)
model.fit(icm)
s = model.similarity
#s.setdiag(0)
r_tag = urm*s
#r_tag = normalize(r_tag, axis=0)
r_tag = bm25_weight(r_tag)
if evaluation: print dr.evaluateMAP(r_tag, verbose = False)
r_tag = dr.reduceRM(r_tag)

0.0406716666667


In [25]:
r_tot = 50*r_al_ar + r_tag 
print dr.evaluateMAP(r_tot, verbose = False)

0.0841366666667


In [56]:
r_tot2 = normalize(r_tot,axis=1) + 0.30* r_al + 0.30 * r_ar
print dr.evaluateMAP(r_tot2, verbose = False)

0.08434775


In [120]:
r_tot3 = 0.80* r_al + 1 * r_ar + 0.10*r_tag 
print dr.evaluateMAP(r_tot3, verbose = False)

0.0838819722222


In [ ]:
#####TUNING#####

In [25]:
# album and artist
def cb_albums_artists(urm,icm,k):
    icm = normalize(icm,axis=0)
    model = BM25Recommender(K=k)
    model.fit(icm)
    s = model.similarity
    r_al_ar = urm*s.T #or s
    r_al_ar = normalize(r_al_ar, axis=0)
    return dr.evaluateMAP(r_al_ar,verbose=False)

In [23]:
#tags
def cb_tags(urm,icm,k):
    icm = normalize(icm,axis=1)   # little improvment? or just overfitting?
    model = TFIDFRecommender(K = k)
    model.fit(icm)
    s = model.similarity
    r_tag = urm*s #######wuuuut no urm
    r_tag = normalize(r_tag, axis=0)
    return dr.evaluateMAP(r_tag, verbose = False)

In [ ]:
#tune tags
urm = dr.getURM_csr()
icm = dr.getICM_csr(albums=False,artists=False,tags=True)

for i in range(5,250,5):
    score= cb_tags(urm=urm,icm=icm,k=i)
    print ("k = {:3d} \t\tmap = {:1.4f}").format(i,score)

In [ ]:
#tune album artist
urm = dr.getURM_csr()
icm = dr.getICM_csr(albums=True,artists=True,tags=False)

for i in range(5,250,5):
    score= cb_albums_artists(urm=urm,icm=icm,k=i)
    print ("k = {:3d} \t\tmap = {:1.4f}").format(i,score)

In [24]:
#tune total
for p in np.arange(0.00, 0.50, 0.025):
    r_tot = p*r_tag + r_al_ar
    score = dr.evaluateMAP(r_tot, verbose = False)
    print ("k = {:1.3f} \t\tmap = {:1.5f}").format(p,score)

k = 0.000 		map = 0.08072
k = 0.025 		map = 0.08129
k = 0.050 		map = 0.08157
k = 0.075 		map = 0.08175
k = 0.100 		map = 0.08220
k = 0.125 		map = 0.08198
k = 0.150 		map = 0.08205
k = 0.175 		map = 0.08216
k = 0.200 		map = 0.08221
k = 0.225 		map = 0.08206
k = 0.250 		map = 0.08192
k = 0.275 		map = 0.08189
k = 0.300 		map = 0.08178
k = 0.325 		map = 0.08162
k = 0.350 		map = 0.08175
k = 0.375 		map = 0.08172
k = 0.400 		map = 0.08167
k = 0.425 		map = 0.08159
k = 0.450 		map = 0.08150
k = 0.475 		map = 0.08143
